In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from re import search
from tqdm  import tqdm
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras import layers
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from PIL import Image
import PIL
import pathlib

from learntools.core import binder
binder.bind(globals())
from learntools.computer_vision.ex5 import *

# Imports
import glob
import shutil
import os, warnings
from matplotlib import gridspec


from tensorflow.keras.preprocessing import image_dataset_from_directory


import cv2
from random import shuffle
plt.style.use('fivethirtyeight')

print("setup completed")

In [ ]:
#run this cell only if the directories exist
shutil.rmtree("./train_images")
shutil.rmtree("./validation_images")
shutil.rmtree("./test_images")

In [ ]:
IMAGE_SIZE = 128
CLASSES = ['0', '1', '2', '3']
LR = 1e-3
DIR = '../input/plant-pathology-2020-fgvc7/images'
TRAIN_DATASET_SIZE = 1100

In [ ]:
train_dataset = pd.read_csv('../input/plant-pathology-2020-fgvc7/train.csv')
print(train_dataset)
test_dataset = pd.read_csv('../input/plant-pathology-2020-fgvc7/test.csv')
print(test_dataset)

# Convert the first image into an array
image = Image.open('../input/plant-pathology-2020-fgvc7/images/Test_0.jpg');
print(np.asarray(image))
img = plt.imshow(image)

In [ ]:
print(train_dataset.head())

In [ ]:
train_dataset['image_id'] = train_dataset['image_id'] + '.jpg'
print(train_dataset.shape)
print(train_dataset)
#define the class names
class_names = train_dataset.loc[:, 'healthy':].columns
print(class_names)

In [ ]:
counter = 0
train_dataset['label'] = 0
for name in class_names:
    train_dataset['label'] = train_dataset['label'] + train_dataset[name] * counter
    counter = counter + 1
    
print(train_dataset)

In [ ]:
# sort the images based on label
train_dataset_healthy = train_dataset[train_dataset.label == 0]
train_dataset_multiple_diseases = train_dataset[train_dataset.label == 1]
train_dataset_rust = train_dataset[train_dataset.label == 2]
train_dataset_scab = train_dataset[train_dataset.label == 3]

In [ ]:
print(train_dataset_healthy)
print(train_dataset_multiple_diseases)
print(train_dataset_rust)
print(train_dataset_scab)

In [ ]:
image_h = Image.open('../input/plant-pathology-2020-fgvc7/images/Train_4.jpg');
#print(np.asarray(image))
img_h = plt.imshow(image_h)

In [ ]:
image_md = Image.open('../input/plant-pathology-2020-fgvc7/images/Train_1.jpg');
#print(np.asarray(image))
img_md = plt.imshow(image_md)

In [ ]:
image_r = Image.open('../input/plant-pathology-2020-fgvc7/images/Train_3.jpg');
#print(np.asarray(image))
img_r = plt.imshow(image_r)

In [ ]:
image_s = Image.open('../input/plant-pathology-2020-fgvc7/images/Train_0.jpg');
#print(np.asarray(image))
img_s = plt.imshow(image_s)

# Get to work

In [ ]:
def get_label_img(img):
    if search("Train", img):
        label = train_dataset.loc[train_dataset["image_id"] == img]['label']
        return label

In [ ]:
def create_train_data():
    count = 0
    for img in tqdm(os.listdir(DIR)):
        label = get_label_img(img)
        path = os.path.join(DIR, img)
        #image_name = img
        #img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMAGE_SIZE, IMAGE_SIZE))
        
        if search("Train", img):
            if count < TRAIN_DATASET_SIZE and label.item() == 0:
                shutil.copy(path, './train_images/healthy')
            elif count < TRAIN_DATASET_SIZE and label.item() == 1:
                shutil.copy(path, './train_images/multiple_diseases')
            elif count < TRAIN_DATASET_SIZE and label.item() == 2:
                shutil.copy(path, './train_images/scab')
            elif count < TRAIN_DATASET_SIZE and label.item() == 3:
                shutil.copy(path, './train_images/rust')
            elif count >= TRAIN_DATASET_SIZE and label.item() == 0:
                shutil.copy(path, './validation_images/healthy')
            elif count >= TRAIN_DATASET_SIZE and label.item() == 1:
                shutil.copy(path, './validation_images/multiple_diseases')
            elif count >= TRAIN_DATASET_SIZE and label.item() == 2:
                shutil.copy(path, './validation_images/scab')
            elif count >= TRAIN_DATASET_SIZE and label.item() == 3:
                shutil.copy(path, './validation_images/rust')
            count = count + 1
        elif search("Test", img):
            shutil.copy(path, './test_images/images') 

In [ ]:
shutil.os.mkdir('./train_images')
shutil.os.mkdir('./test_images')
shutil.os.mkdir('./test_images/images')
shutil.os.mkdir('./validation_images')
shutil.os.mkdir('./train_images/healthy')
shutil.os.mkdir('./train_images/multiple_diseases')
shutil.os.mkdir('./train_images/scab')
shutil.os.mkdir('./train_images/rust')
shutil.os.mkdir('./validation_images/healthy')
shutil.os.mkdir('./validation_images/multiple_diseases')
shutil.os.mkdir('./validation_images/scab')
shutil.os.mkdir('./validation_images/rust')
train_data = create_train_data()


In [ ]:
filenames = glob.glob("./validation_images/rust/*.jpg")
filenames.sort()
filenames = glob.glob("./validation_images/healthy/*.jpg")
filenames.sort()
filenames = glob.glob("./validation_images/scab/*.jpg")
filenames.sort()
filenames = glob.glob("./validation_images/multiple_diseases/*.jpg")
filenames.sort()


Old model accuracy: 66%

import tensorflow.keras.layers.experimental.preprocessing as preprocessing
# keras
model = keras.Sequential([
    preprocessing.RandomFlip('horizontal'), # flip left-to-right
    preprocessing.RandomContrast(0.5),
    # Block One
    layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same',
                  input_shape=[IMAGE_SIZE, IMAGE_SIZE, 3]),
    layers.MaxPool2D(2, 2),
    layers.Dropout(0.1),

    # Block Two
    layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(2, 2),
    layers.Dropout(0.3),

    # Block Three
    layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    #layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(2, 2),
    layers.Dropout(0.3),


    # Head
    layers.Flatten(),
    layers.Dense(128, activation='selu'),
    layers.Dense(4, activation='softmax'),
])


import tensorflow.keras.layers.experimental.preprocessing as preprocessing
# keras
model = keras.Sequential([
    preprocessing.RandomFlip('horizontal'), # flip left-to-right
    preprocessing.RandomContrast(0.5),
    # Block One
    layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same',
                  input_shape=[IMAGE_SIZE, IMAGE_SIZE, 3]),
    layers.MaxPool2D(pool_size=4,
                     strides=2,
                     padding='same'),
    layers.Dropout(0.2),

    # Block Two
    layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(pool_size=4,
                     strides=2,
                     padding='same'),
    layers.Dropout(0.3),

    # Block Three
    layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(pool_size=4,
                     strides=2,
                     padding='same'),
    layers.Dropout(0.5),


    # Head
    layers.Flatten(),
    layers.Dense(128, activation='selu'),
    layers.Dense(4, activation='softmax'),
])

New model

In [ ]:
import tensorflow.keras.layers.experimental.preprocessing as preprocessing
# keras
model = keras.Sequential([
    preprocessing.RandomFlip('horizontal'), # flip left-to-right
    preprocessing.RandomContrast(0.5),
    # Block One
    layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same',
                  input_shape=[IMAGE_SIZE, IMAGE_SIZE, 3]),
    layers.MaxPool2D(pool_size=4,
                     strides=2,
                     padding='same'),
    layers.Dropout(0.2),

    # Block Two
    layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(pool_size=4,
                     strides=2,
                     padding='same'),
    layers.Dropout(0.4),

    # Block Three
    layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(pool_size=4,
                     strides=2,
                     padding='same'),

    # Head
    layers.Flatten(),
    layers.Dense(128, activation='selu'),
    layers.Dense(4, activation='softmax'),
])


In [ ]:
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
#set_seed()

In [ ]:
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore") # to clean up output cells


# Load training and validation sets
ds_train_ = image_dataset_from_directory(
    './train_images',
    labels='inferred',
    label_mode='int',
    image_size=[IMAGE_SIZE, IMAGE_SIZE],
    interpolation='nearest',
    batch_size=64,
)
ds_valid_ = image_dataset_from_directory(
    './validation_images',
    labels='inferred',
    label_mode='int',
    image_size=[IMAGE_SIZE, IMAGE_SIZE],
    interpolation='nearest',
    batch_size=64,
)

ds_test_ = image_dataset_from_directory(
    './test_images',
    image_size=[IMAGE_SIZE, IMAGE_SIZE],
    label_mode=None
)

# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

def convert_to_float_test(image):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image



In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_test = (
    ds_test_
    .map(convert_to_float_test)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
print(type(ds_test))
print(type(ds_train))

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(epsilon=0.01, learning_rate=LR),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=50
)

In [ ]:
model.summary()

In [ ]:
import pandas as pd
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot();

In [ ]:
probabilities = model.predict(ds_valid)
predictions = np.argmax(probabilities, axis=-1)
print(predictions)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

In [ ]:
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer
auc = multiclass_roc_auc_score(train_dataset.iloc[TRAIN_DATASET_SIZE:,[5]], predictions)
print('AUC: %.2f' % auc)


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
conf_matrix = confusion_matrix(train_dataset.iloc[TRAIN_DATASET_SIZE:,[5]], predictions, labels=[0, 1, 2, 3])


In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(conf_matrix, classes=[0, 1, 2, 3],
                      title='Confusion matrix')

In [ ]:
print(("Best Validation Loss: {:0.4f}" +\
      "\nBest Validation Accuracy: {:0.4f}")\
      .format(history_frame['val_loss'].min(), 
              history_frame['val_accuracy'].max()))